<a href="https://colab.research.google.com/github/tararajagopalan/WebsiteQueryingProject/blob/main/GeneratingTextSummaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#using the google api using the command below:
from google.colab import userdata

#install api key from secretes and get it from there
#gemini api key is below
api_key = userdata.get('GOOGLE_API_KEY')

#DO NOT PRINT api_key: will leak it!

from google import genai
from google.genai import types
import pathlib
import httpx

#the gemini api is running on a server. the gemini api key is used to COMMUNICATE with this gemini api server.
#1) authentication: the gemini api key authenticates who you are to the server
#2) authorization: the gemini api key makes sure that you are allowed to call these functions
#client is using the gemini api_key
gemini_client = genai.Client(api_key=api_key)

In [ ]:

def getVideoSummary(VideoFilePath,client):
  """
    Generates a short caption for video files
    Only for videos of size <20Mb
    """

  #we are passing in the video file path, reading the contents of the file into video bytes
  video_file_path =  VideoFilePath
  video_bytes = open(video_file_path, 'rb').read()

  #client: has gemini api key to access gemini api server and is a client for the GEMINI REST API where it can access gemini functions from the REST API
  #client is sending a request to the server (the REST API) calling the generate_content function (a function in the  Gemini API)
  #generate_content parameters: specifies model as gemini-2.5-flash,
    #inside of contents parameter, specifies type of file and send data via video bytes, and gives a prompt on what to do
  #inside the response parameter, it gets back the 3 sentence summary
  response = client.models.generate_content(
    model='models/gemini-2.5-flash',
    contents=types.Content(
        parts=[
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            ),
            types.Part(text='Please summarize the video in 3 sentences.')
        ]
    )
 )
  return response.text




In [ ]:

def getPDFSummary(PDFFilePath,client):
   """
    Generates a short caption for PDF.
    """

  #Retrieve and encode the PDF byte
  file_path = pathlib.Path(PDFFilePath)

  #Upload the pdf using the File API
  sample_file = client.files.upload(
    file = file_path,
  )

  prompt = "Summarize the Document"

  #client is sending a request to the REST API server and wants to use the generate_content function on the REST API SERVER
  #parameters: model (gemini-2.5-flash),contents has the pdf file to pass in and prompt of what we want to do
  response = client.models.generate_content(
      model = "gemini-2.5-flash",
      contents = [sample_file,"Summarize the Document"]
  )
  #response CONTAINS THE SUMMARY and by saying 'return response.text' we are saying to only return the text portion of the response
  return response.text




In [ ]:
def getImageSummary(ImageFilePath, client, max_words=30):
    """
    Generates a short caption for an image.
    max_words = the maximum number of words in the returned caption.
    """

    # Read image bytes
    with open(ImageFilePath, 'rb') as f:
        image_bytes = f.read()

    prompt = f"Caption this image in under {max_words} words, concise and vivid."

    # Send request with a prompt that explicitly asks for brevity
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            types.Part.from_bytes(
                data=image_bytes,
                mime_type="image/jpeg"
            ),
            prompt
            #f"Caption this image in under {max_words} words, concise and vivid."
        ]
    )

    return response.text.strip()


In [ ]:
def getTextSummary(txtFilePath, client):
    """
    Generates a short summary of a text file.
    """

    with open(txtFilePath, 'r') as f:
        text = f.read()
    prompt = "Summarize the Document"

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            types.Part(text=text),
            prompt
        ]

    )

    return response.text


In [ ]:
#Function of getfileSummary to get the summary for the designated file passed in
#calls other helper functions that were written above

def getFileSummary(FileType, FilePath,client):
  if FileType == '.mp4':
    return getVideoSummary(FilePath,client)
  elif FileType == '.pdf':
    return getPDFSummary(FilePath,client)
  elif FileType == '.jpg' or FileType == ".jpeg" or FileType == ".webp":
    return getImageSummary(FilePath,client)
  elif FileType == '.txt':
    return getTextSummary(FilePath,client)
  else:
    return "Invalid File Type"

In [ ]:
#milvus installations are below:
!pip install -U pymilvus
!pip install --upgrade pymilvus
!pip install "pymilvus[model]"

In [ ]:
from pymilvus import model

In [ ]:
#SETTING UP VECTOR DATA BASE: creating a database on the mounted google drive
#sets up data base in the project_work.db file on google drive

#declaring the client here and connecting it to project_work database!
from pymilvus import MilvusClient

milvus_client= MilvusClient("/content/drive/MyDrive/project_work.db")

collection_name = "project_collection"

In [ ]:
import os
import time
def getInformationForEachFile(folderPath,gemini_client,milvus_client):

  folderPathArray = []

  for item_name in os.listdir(folderPath):
        item_path = os.path.join(folderPath, item_name)
        print("Processing: " + item_path)
        #item_path: full path to item (file or folder), combines folder and item_name

        if os.path.isfile(item_path):

            #print(f"File: {item_name}")
            _, extension = os.path.splitext(item_name)
            ext = extension.lower()
            #summary = getFileSummary(extension,item_path,gemini_client)

            if ext in [".jpeg",".webp",".jpg"]:
              summary = getImageSummary(item_path,gemini_client)
            elif ext in [".pdf"]:
              summary = getPDFSummary(item_path,gemini_client)
            elif ext in [".mp4",".mov"]:
              summary = getVideoSummary(item_path,gemini_client)
            elif ext in [".txt"]:
              summary = getTextSummary(item_path,gemini_client)
            else:
              summary = "Invalid File Type"

            embedding_fn = model.DefaultEmbeddingFunction()
            vectors = embedding_fn.encode_documents([summary])

            item_path = item_path.split("Static/",1)[1]

            folderPathArray.append({"text":summary,"embedding":vectors[0], "FilePath":item_path, "FileExtension":ext})
        elif os.path.isdir(item_path):
            print(f"Directory: {item_name}")
            # You can recursively iterate into subdirectories if needed
            # For example: iterate_folder(item_path)
        time.sleep(5)  # Pause for 5 seconds
        print("5 seconds have passed.")
  return folderPathArray

In [ ]:
#function to insert summaries into the client collection
def insertDataIntoCollection(folderpath, gemini_client, milvus_client, collection_name):

  data = getInformationForEachFile(folderpath,gemini_client,milvus_client)

  print(data)

  #uses milvus client to insert data into the appropriate collection for the project (project_collection)
  res = milvus_client.insert(collection_name=collection_name,data=data)
  print(res)


In [ ]:
#testing insertDataIntoCollection function for IMAGE SUMMARIES: WORKS!!

folderPath = '/content/drive/My Drive/Flask/Static/Images/Samples'
insertDataIntoCollection(folderPath,gemini_client,milvus_client,collection_name)

In [ ]:
#testing insertDataIntoCollection function for PDF SUMMARIES: WORKS!!

folderPathPDF = '/content/drive/My Drive/Flask/Static/PDF/Samples'
insertDataIntoCollection(folderPathPDF,gemini_client,milvus_client,collection_name)

In [ ]:
#testing insertDataIntoCollection function for Video Summaries: PENDING
folderPathVideos = '/content/drive/My Drive/Flask/Static/Videos/Samples'
insertDataIntoCollection(folderPathVideos,gemini_client,milvus_client,collection_name)

In [ ]:
#testing insertDataIntoCollection function for Text Summaries: WORKS!!
folderPathText = '/content/drive/My Drive/Flask/Static/Text/Samples'
insertDataIntoCollection(folderPathText,gemini_client,milvus_client,collection_name)